In [1]:
import xarray as xr
import pandas as pd
import numpy as np
from heapq import heappush, heappop

In [2]:

def download_gfs_data():
    """Download U and V wind components at 850mb from NOMADS server"""
    # Update the URL with the latest available GFS data
    current_time = pd.Timestamp.now('UTC').tz_convert(None)
    gfs_init = current_time - pd.Timedelta('5h')
    gfs_hour = np.floor(gfs_init.hour//6)*6
    url = f"https://nomads.ncep.noaa.gov/dods/gfs_0p25/gfs{gfs_init.year}{gfs_init.month:02}{gfs_init.day:02}/gfs_0p25_{gfs_hour:02}z"
    
    try:
        ds = xr.open_dataset(url)
        u = ds['ugrdprs'].sel(lev=850, time=current_time, method='nearest')
        v = ds['vgrdprs'].sel(lev=850, time=current_time, method='nearest')
        return u, v, u.lat.values, u.lon.values
    except Exception as e:
        print(f"Error downloading data: {e}")
        raise


In [3]:
def calculate_time_cost(u, v, lat, lon, start_idx, end_idx, airspeed):
    """Calculate time cost between two grid points considering wind components"""
    i1, j1 = start_idx
    i2, j2 = end_idx
    
    # Calculate distance components in meters
    dlat = (lat[i2] - lat[i1]) * 111320  # 1 degree ≈ 111,320 meters
    dlon = (lon[j2] - lon[j1]) * 111320 * np.cos(np.radians(lat[i1]))
    distance = np.hypot(dlon, dlat)
    
    if distance == 0:
        return 0
    
    # Wind components at starting point
    u_wind = u[i1, j1].item()
    v_wind = v[i1, j1].item()
    
    # Wind component in direction of motion
    wind_component = (u_wind * dlon + v_wind * dlat) / distance
    effective_speed = airspeed + wind_component
    
    if effective_speed <= 0:
        return np.inf  # Unpassable edge
    
    return distance / effective_speed

In [4]:
def dijkstra(u, v, lat, lon, start_idx, end_idx, airspeed=25):
    """Find fastest path using Dijkstra's algorithm considering wind effects"""
    rows, cols = len(lat), len(lon)
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1),
                  (-1, -1), (-1, 1), (1, -1), (1, 1)]  # 8 directions
    
    heap = []
    heappush(heap, (0, start_idx[0], start_idx[1]))
    
    costs = np.full((rows, cols), np.inf)
    costs[start_idx] = 0
    
    predecessors = {}
    visited = set()

    while heap:
        current_cost, i, j = heappop(heap)
        
        if (i, j) in visited:
            continue
        visited.add((i, j))
        
        if (i, j) == end_idx:
            break
            
        for di, dj in directions:
            ni, nj = i + di, j + dj
            if 0 <= ni < rows and 0 <= nj < cols:
                if (ni, nj) in visited:
                    continue
                
                edge_cost = calculate_time_cost(u, v, lat, lon, (i, j), (ni, nj), airspeed)
                new_cost = current_cost + edge_cost
                
                if new_cost < costs[ni, nj]:
                    costs[ni, nj] = new_cost
                    predecessors[(ni, nj)] = (i, j)
                    heappush(heap, (new_cost, ni, nj))
    
    # Reconstruct path
    path = []
    current = end_idx
    while current in predecessors:
        path.append(current)
        current = predecessors[current]
    path.append(start_idx)
    return path[::-1], costs[end_idx]

In [5]:
# Download GFS data
u, v, lat, lon = download_gfs_data()

/home/travis/mambaforge/envs/general2/lib/python3.13/site-packages/xarray/coding/times.py:206: SerializationWarning: Ambiguous reference date string: 1-1-1 00:00:0.0. The first value is assumed to be the year hence will be padded with zeros to remove the ambiguity (the padded reference date string is: 0001-1-1 00:00:0.0). To remove this message, remove the ambiguity by padding your reference date strings with zeros.
  ref_date = _ensure_padded_year(ref_date)


In [6]:
u.load()

<xarray.DataArray 'ugrdprs' (lat: 721, lon: 1440)> Size: 4MB
array([[-3.6989453, -3.6989453, -3.6989453, ..., -3.6989453, -3.6989453,
        -3.6989453],
       [-3.6689453, -3.6689453, -3.6689453, ..., -3.6689453, -3.6689453,
        -3.6689453],
       [-3.7589452, -3.7589452, -3.7489452, ..., -3.7689452, -3.7689452,
        -3.7589452],
       ...,
       [ 8.601054 ,  8.581055 ,  8.551055 , ...,  8.661055 ,  8.641054 ,
         8.621055 ],
       [ 6.5910544,  6.5810547,  6.5710545, ...,  6.6210546,  6.6110544,
         6.6010547],
       [ 6.7210546,  6.7110543,  6.7010546, ...,  6.751055 ,  6.7410545,
         6.7310543]], shape=(721, 1440), dtype=float32)
Coordinates:
    time     datetime64[ns] 8B 2025-04-03
    lev      float64 8B 850.0
  * lat      (lat) float64 6kB -90.0 -89.75 -89.5 -89.25 ... 89.5 89.75 90.0
  * lon      (lon) float64 12kB 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
Attributes:
    long_name:  ** (1000 975 950 925 900.. 10 7 4 2 1) u-component of wind [m...

In [7]:
v.load()

<xarray.DataArray 'vgrdprs' (lat: 721, lon: 1440)> Size: 4MB
array([[ 0.19791014,  0.17791015,  0.16791016, ...,  0.24791016,
         0.22791015,  0.21791016],
       [-0.21208984, -0.23208983, -0.24208984, ..., -0.15208983,
        -0.17208984, -0.18208984],
       [-0.38208982, -0.40208983, -0.42208984, ..., -0.32208985,
        -0.34208983, -0.36208984],
       ...,
       [-2.8220897 , -2.85209   , -2.8820899 , ..., -2.7320898 ,
        -2.7620897 , -2.7920897 ],
       [-2.07209   , -2.09209   , -2.1220899 , ..., -1.9920897 ,
        -2.0120897 , -2.0420897 ],
       [-2.2820897 , -2.3120897 , -2.34209   , ..., -2.2020898 ,
        -2.2320898 , -2.2520897 ]], shape=(721, 1440), dtype=float32)
Coordinates:
    time     datetime64[ns] 8B 2025-04-03
    lev      float64 8B 850.0
  * lat      (lat) float64 6kB -90.0 -89.75 -89.5 -89.25 ... 89.5 89.75 90.0
  * lon      (lon) float64 12kB 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
Attributes:
    long_name:  ** (1000 975 950 925 900.. 10 7 4 2 1) v-component of wind [m...

In [8]:
# Set start and end points (indices in the grid)
start_idx = (50, 100)  # Example indices - adjust according to your domain
end_idx = (60, 120)    # Example indices - adjust according to your domain

# Find fastest path
path, total_time = dijkstra(u, v, lat, lon, start_idx, end_idx)

In [9]:

if path:
    print(f"Fastest path found with total time: {total_time/3600:.2f} hours")
    print("Path indices (lat, lon):")
    for point in path:
        print(f"({lat[point[0]]:.2f}, {lon[point[1]]:.2f})")
else:
    print("No path found")

Fastest path found with total time: 4.03 hours
Path indices (lat, lon):
(-77.50, 25.00)
(-77.50, 25.25)
(-77.50, 25.50)
(-77.50, 25.75)
(-77.50, 26.00)
(-77.50, 26.25)
(-77.50, 26.50)
(-77.50, 26.75)
(-77.50, 27.00)
(-77.50, 27.25)
(-77.50, 27.50)
(-77.25, 27.75)
(-77.00, 28.00)
(-76.75, 28.25)
(-76.50, 28.50)
(-76.25, 28.75)
(-76.00, 29.00)
(-75.75, 29.25)
(-75.50, 29.50)
(-75.25, 29.75)
(-75.00, 30.00)
